In [24]:
import numpy as np
import sqlite3
import pandas as pd
import csv
from datetime import datetime, timedelta
import time
import datetime
import requests
from io import StringIO

In [25]:
save_list=np.load("save_list.npy")
now_list=np.load("now.npy")

In [27]:
## Open file   把當月份的代號txt  拿出來
fp = open('911.txt', "r")
line = fp.readline()
month_predict=[]
## 用 while 逐行讀取檔案內容，直至檔案結尾
while line:
#    print (line)
    line=line.replace(" ","")
    month_predict.append(line.replace("\n",""))
    line = fp.readline()
fp.close()

In [28]:
# 抓上市公司股價
def stock_value(datestr):
    #把當月11號的台股資訊 更新
    #datestr = '20200911'
    # 下載股價
    r = requests.post('https://www.twse.com.tw/exchangeReport/MI_INDEX?response=csv&date=' + datestr + '&type=ALL')
    # 整理資料，變成表格
    df = pd.read_csv(StringIO(r.text.replace("=", "")), 
                header=["證券代號" in l for l in r.text.split("\n")].index(True)-1)
    # 整理一些字串：
    df = df.apply(lambda s: pd.to_numeric(s.astype(str).str.replace(",", "").replace("+", "1").replace("-", "-1"), errors='coerce'))
    # 顯示出來
    #df.head()
    return df

In [29]:
#抓上櫃公司股價
def stock_value2(datestr):
    link = 'http://www.tpex.org.tw/web/stock/aftertrading/daily_close_quotes/stk_quote_download.php?l=zh-tw&d='+datestr+'&s=0,asc,0'
    r = requests.get(link)
    r.ok
    lines = r.text.replace('\r', '').split('\n')
    df = pd.read_csv(StringIO("\n".join(lines[3:])), header=None)
    #df.head()
    df.columns = list(map(lambda l: l.replace(' ',''), lines[2].split(',')))
    #df.index = df['代號']
    #df = df.drop(['代號'], axis=1)
    #df.head()
    return df


In [30]:
predict_day='20200911'
now_day="20200924"

In [31]:
df_last_month=stock_value('20200911')
df_now=stock_value('20200925')

In [32]:
df2_last_month=stock_value2("109/09/11")
df2_now=stock_value2('109/09/25')

In [33]:
month_predict

['2884',
 '2887',
 '2885',
 '8111',
 '2206',
 '3285',
 '1477',
 '3272',
 '5471',
 '8499',
 '1589',
 '4906',
 '3661',
 '8938',
 '4977',
 '3163',
 '2352']

In [34]:
df=df_now
for i in range(len(month_predict)):
    a=df[pd.to_numeric(df['證券代號'], errors='coerce') == float(month_predict[i])]
    try:
        b=a.values.tolist()[0]
        print(b[0])
    except:
        pass

2884.0
2887.0
2885.0
2206.0
1477.0
5471.0
8499.0
1589.0
4906.0
3661.0
4977.0
2352.0


In [35]:
#拿上市收盤價資料
def take_value(df,id):
    take_info=df[pd.to_numeric(df['證券代號'], errors='coerce') == id]
    value=float(take_info["收盤價"].values)
    return value
#拿上櫃收盤價資料
def take_value2(df,id):
    take_info=df[pd.to_numeric(df['代號'], errors='coerce') == id]
    value=float(take_info["收盤"].values)
    return value

In [36]:
#把預測的股票價錢都拿出來
now_list=[]
for i in range(len(month_predict)):
    temp=[]
    index=float(month_predict[i])
    #print(type(index),index)
    try:
        start_value=take_value(df_last_month,index)
        end_value=take_value(df_now,index)
    except:
        start_value=take_value2(df2_last_month,index)
        end_value=take_value2(df2_now,index)
    #print(i,str(index).replace(".0",""),start_value,end_value)
    temp.append(str(index).replace(".0",""))
    temp.append("")
    temp.append(start_value)
    temp.append(end_value)
    now_list.append(temp)

In [37]:
#得到代號 空白   當月11號價格   現在價格
now_list

[['2884', '', 26.7, 25.3],
 ['2887', '', 13.1, 12.5],
 ['2885', '', 18.3, 17.3],
 ['8111', '', 16.8, 15.0],
 ['2206', '', 23.2, 22.35],
 ['3285', '', 24.45, 22.6],
 ['1477', '', 188.0, 167.5],
 ['3272', '', 60.0, 58.1],
 ['5471', '', 53.9, 67.3],
 ['8499', '', 132.5, 129.5],
 ['1589', '', 99.6, 89.0],
 ['4906', '', 26.55, 25.25],
 ['3661', '', 411.0, 483.5],
 ['8938', '', 38.5, 36.55],
 ['4977', '', 113.0, 101.5],
 ['3163', '', 61.6, 63.0],
 ['2352', '', 19.3, 18.1]]

In [66]:
#查詢list in str
#any('1101' in item for item in a)
#查詢list in str
def search_name(name):
    company=[]
    df = pd.read_csv('公司/上市.csv')  
    a=df["有價證券代號及名稱"].tolist()
    df = pd.read_csv('公司/上櫃.csv')  
    b=df["有價證券代號及名稱"].tolist()
    df = pd.read_csv('公司/興櫃.csv')  
    c=df["有價證券代號及名稱"].tolist()
    for i in a:
        if i.__contains__(name) :
            company=i
            #print(i, " is containing")
    for i in b:
        if i.__contains__(name) :
            company=i
            #print(i, " is containing")
    for i in c:
        if i.__contains__(name) :
            company=i
            #print(i, " is containing")
    return company

In [38]:
#不要亂執行   拿來刪檔案用得
db =  sqlite3.connect('db.sqlite3')
##db.execute("INSERT INTO type_data (tag)   VALUES ('{}')".format(data))
db.execute("delete from basicHistory")
db.commit()
db =  sqlite3.connect('db.sqlite3')
##db.execute("INSERT INTO type_data (tag)   VALUES ('{}')".format(data))
#db.execute("delete from technicHistory")
db.commit()

In [76]:
def change_parameter(save_list):
    final_update=now_day[:4]+"-"+now_day[4:6]+"-"+now_day[6:8]
    name=search_name(now_list[i][0]).replace("\u3000", " ")
    start_date=predict_day[:4]+"-"+predict_day[4:6]+"-"+predict_day[6:8]
    start_price=now_list[i][2]
    over_date="2020-10-10"
    current_price=now_list[i][3]

    now_return=(float(current_price)-float(start_price))/float(start_price)
    now_return= round(now_return*100,2)

    if float(now_return)>0:
        types="+"
    else:
        types="-"
    return final_update,name,start_date,start_price,over_date,current_price,now_return,types

In [77]:
change_parameter(now_list[1])

('2020-09-24', '2352 佳世達', '2020-09-11', 19.3, '2020-10-10', 18.1, -6.22, '-')

In [79]:
# 更新月營收策略  當月股價
db =  sqlite3.connect('db.sqlite3')
#db.execute("INSERT INTO type_data (tag)   VALUES ('{}')".format(data))
db.execute("delete from basicCurrent")
db.commit()
pk=1
for i in range(len(now_list)):
    final_update,name,start_date,start_price,over_date,current_price,now_return,types=change_parameter(now_list[i]) 
    db =  sqlite3.connect('db.sqlite3')
    db.execute("INSERT INTO basicCurrent (id,final_update,stock_name,start_date,start_price,over_date,current_price,now_return,type)   VALUES ('{}','{}','{}','{}','{}','{}','{}','{}','{}')".format(pk,final_update,name,start_date,start_price,over_date,current_price,now_return,types))
    db.commit()
    db.close()
    pk=pk+1

In [6]:
#基本面歷史
pk=1
for i in range(len(save_list)):
    name=search_name(save_list[i][0]).replace("\u3000", " ")
    start_date=save_list[i][1]
    over_date=save_list[i][2]
    buy_price=save_list[i][3]
    sell_price=save_list[i][4]
    return_value=(float(sell_price)-float(buy_price))/float(buy_price)
    return_value= round(return_value*100,2)
    if return_value>0:
        types="+"
    else:
        types="-"

    db =  sqlite3.connect('db.sqlite3')
    db.execute("INSERT INTO basicHistory (id,stock_name,start_date,buy_price,over_date,sell_price,return_value,type)   VALUES ('{}','{}','{}','{}','{}','{}','{}','{}')".format(pk,name,start_date,buy_price,over_date,sell_price,return_value,types))
    db.commit()
    db.close()
    pk=pk+1

In [7]:
#技術面歷史
pk=1
for i in range(len(save_list)):
    name=search_name(save_list[i][0]).replace("\u3000", " ")
    start_date=save_list[i][1]
    over_date=save_list[i][2]
    buy_price=save_list[i][3]
    sell_price=save_list[i][4]
    return_value=(float(sell_price)-float(buy_price))/float(buy_price)
    return_value= round(return_value*100,2)
    if return_value>0:
        types="+"
    else:
        types="-"

    db =  sqlite3.connect('db.sqlite3')
    db.execute("INSERT INTO technicHistory (id,stock_name,start_date,buy_price,over_date,sell_price,return_value,type)   VALUES ('{}','{}','{}','{}','{}','{}','{}','{}')".format(pk,name,start_date,buy_price,over_date,sell_price,return_value,types))
    db.commit()
    db.close()
    pk=pk+1

In [40]:
db =  sqlite3.connect('db.sqlite3')
#db.execute("INSERT INTO type_data (tag)   VALUES ('{}')".format(data))
db.execute("delete from basicCurrent")
db.commit()
db =  sqlite3.connect('db.sqlite3')
#db.execute("INSERT INTO type_data (tag)   VALUES ('{}')".format(data))
#db.execute("delete from technicCurrent")
#db.commit()

In [ ]:
db =  sqlite3.connect('db.sqlite3')
#db.execute("INSERT INTO type_data (tag)   VALUES ('{}')".format(data))
db.execute("delete from basicCurrent")
db.commit()

In [10]:
#基本面現在
pk=1
for i in range(len(now_list)):
    final_update="2020-07-14"
    name=search_name(now_list[i][0]).replace("\u3000", " ")
    start_date="2020-07-10"
    start_price=now_list[i][2]
    over_date="2020-08-14"
    current_price=now_list[i][3]
    now_return=now_list[i][1]
    if float(now_return)>0:
        types="+"
    else:
        types="-"
        
        
    db =  sqlite3.connect('db.sqlite3')
    db.execute("INSERT INTO basicCurrent (id,final_update,stock_name,start_date,start_price,over_date,current_price,now_return,type)   VALUES ('{}','{}','{}','{}','{}','{}','{}','{}','{}')".format(pk,final_update,name,start_date,start_price,over_date,current_price,now_return,types))
    db.commit()
    db.close()
    pk=pk+1

In [11]:
#基本面現在
pk=1
for i in range(len(now_list)):
    final_update="2020-07-14"
    name=search_name(now_list[i][0]).replace("\u3000", " ")
    start_date="2020-07-10"
    start_price=now_list[i][2]
    over_date="2020-08-14"
    current_price=now_list[i][3]
    now_return=now_list[i][1]
    if float(now_return)>0:
        types="+"
    else:
        types="-"
        
        
    db =  sqlite3.connect('db.sqlite3')
    db.execute("INSERT INTO technicCurrent (id,final_update,stock_name,start_date,start_price,over_date,current_price,now_return,type)   VALUES ('{}','{}','{}','{}','{}','{}','{}','{}','{}')".format(pk,final_update,name,start_date,start_price,over_date,current_price,now_return,types))
    db.commit()
    db.close()
    pk=pk+1